In [11]:
import re
import os
import cv2
import numpy as np
import pandas as pd

from insightface.app import FaceAnalysis

## Load Model

In [12]:
# configure face analysis
faceapp = FaceAnalysis(
    name='buffalo_sc',
    root='insightface_model',
    providers=["CPUExecutionProvider"]
)

faceapp.prepare(ctx_id=0, det_size=(640, 640), det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


## Extract Facial Features and labels

In [18]:
def clean_name(string):
    string = re.sub(r'[^a-zA-Z0-9\s]', ' ', string)
    return string.title()

person_info = []
for folder_name in os.listdir(path='../images'):
    role, name = folder_name.split('-')
    name = clean_name(name)
    role = clean_name(role)
    print(f'Processing {role} - {name}')
    for image_name in os.listdir(path=f'../images/{folder_name}'):
        image_path = f'../images/{folder_name}/{image_name}'
        
        img_arr = cv2.imread(image_path)

        result = faceapp.get(img=img_arr)

        if len(result) > 0:

            res = result[0]
            embedding = res["embedding"]
            person_info.append([name, role, image_name, embedding])

dataframe = pd.DataFrame(person_info, columns=['name', 'role', 'image', 'embedding'])
dataframe # store this in redis

Processing Student - Filip Shramko
Processing Student - Mateusz Polis
Processing Student - Patryk Sukiennik
Processing Student - Sebastian Rydz
Processing Teacher - Mateusz Osik


,name,role,image,embedding
0,Filip Shramko,Student,filip_shramko_1.jpg,"[-1.5434499, -1.1279633, 0.958638, 0.57546866,..."
1,Filip Shramko,Student,filip_shramko_2.jpg,"[-1.8280271, -0.74998987, 1.6958079, -0.425696..."
2,Filip Shramko,Student,filip_shramko_3.jpg,"[-1.4542996, -1.2352338, 0.959083, 0.068062484..."
3,Filip Shramko,Student,filip_shramko_4.jpg,"[-1.915733, 2.2441149e-05, 1.334084, 0.5913815..."
4,Filip Shramko,Student,filip_shramko_5.jpg,"[-2.0738428, -0.95282745, 0.13931799, -0.02284..."
5,Filip Shramko,Student,filip_shramko_6.jpg,"[-2.3037813, -0.64571285, 2.0764215, 0.180688,..."
6,Mateusz Polis,Student,mateusz_polis_1.jpg,"[-1.1474814, 2.1227167, 0.20061958, 0.90253866..."
7,Mateusz Polis,Student,mateusz_polis_2.jpg,"[-0.12210239, 0.5270046, -0.10220441, 0.512440..."
8,Mateusz Polis,Student,mateusz_polis_3.jpg,"[-2.5321522, 1.8216596, 1.0976498, 0.58796716,..."
9,Mateusz Polis,Student,mateusz_polis_4.jpg,"[-0.24887145, 0.70079684, -0.5873411, 0.036971..."
